In [24]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Sequential, Model, load_model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
#resie all the images to thin
image_size = [224, 224]

In [4]:
train_path = "Datasets/Train"
valid_path = "Datasets/Test"

In [5]:
#add preprocessing layer to the front of vgg
vgg = VGG16(input_shape=image_size+[3], weights="imagenet", include_top=False)

58892288/58889256 [==============================] - 38s 1us/step


In [6]:
#don't train existing weights
for layer in vgg.layers:
    layer.trainable=False

In [8]:
#useful for getting number of classes
folders = glob("Datasets/Train/*")

In [9]:
#our layers
x = Flatten()(vgg.output)

In [10]:
prediction = Dense(len(folders), activation="softmax")(x)

In [11]:
#create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [12]:
#view the structure of the model
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [14]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
#image_data_augmentation

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255, shear_range=0.2,
                            zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1/255)

training_set = train_datagen.flow_from_directory("Datasets/Train", batch_size=32,
                                                 target_size=(224, 224), class_mode="categorical")

test_set = test_datagen.flow_from_directory("Datasets/Test", batch_size=32,
                                                 target_size=(224, 224), class_mode="categorical")

Found 0 images belonging to 4 classes.
Found 0 images belonging to 4 classes.


In [19]:
#fit the model
r = model.fit_generator(training_set, validation_data=test_set, epochs=5,
                        steps_per_epoch=len(training_set), validation_steps=len(test_set))

ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [20]:
#loss_plot
plt.plot(r.history["loss"], label="Training Loss")

NameError: name 'r' is not defined

In [21]:
#loss_plot
plt.plot(r.history["val_loss"], label="Validation Loss")

NameError: name 'r' is not defined

In [22]:
#accuracy_plot
plt.plot(r.history["accuracy"], label="Training Accuracy")

NameError: name 'r' is not defined

In [23]:
#accuracy_plot
plt.plot(r.history["val_accuracy"], label="Validation Accuracy")

NameError: name 'r' is not defined

In [26]:
model.save("new_model.h5")